<a href="https://colab.research.google.com/github/sonalgan/IndiSpeak/blob/main/indispeak_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Installation of packages
!sudo apt-get install portaudio19-dev -y
!pip install --upgrade numpy==1.21.6 resampy==0.2.2 tensorflow==2.8.2 tf_slim==1.1.0 six soundfile sounddevice  ipywidgets pyaudio pydub torchaudio

!git clone https://github.com/tensorflow/models.git
# Grab the VGGish model
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz
# Copy the source files to the current directory.
!cp models/research/audioset/vggish/* .
!rm -rf models

!git clone https://github.com/sonalgan/IndiSpeak.git
!cp IndiSpeak/* .
!cp IndiSpeak/trained_models/*  .
!rm -rf IndiSpeak


!pip install ffmpeg-python





Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 9 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (292 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd

Cloning into 'models'...
remote: Enumerating objects: 86835, done.
remote: Counting objects: 100% (589/589), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 86835 (delta 324), reused 542 (delta 293), pack-reused 86246
Receiving objects: 100% (86835/86835), 598.93 MiB | 26.46 MiB/s, done.
Resolving deltas: 100% (62206/62206), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  77.1M      0  0:00:03  0:00:03 --:--:-- 77.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   335k      0 --:--:-- --:--:-- --:--:--  334k
Cloning into 'IndiSpeak'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (48/48), done.
remote: 

In [4]:
## Importing packgaes
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import scipy
import os
import numpy as np
import joblib
import torchaudio
import torch
import ipywidgets as widgets
from IPython.display import Audio, display
from feature_extractor import FeatureExtractor
from pydub import AudioSegment

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
## Record Audio

audio, sr = get_audio()


In [ ]:
##Save audio

scipy.io.wavfile.write('file.mp3', sr, audio)

In [ ]:

def crop_and_save_audio(audio_file, output_file, duration=5000):
    # Load the audio file using torchaudio
    waveform, sample_rate = torchaudio.load(audio_file)

    duration_ms = duration

    audio_segment = AudioSegment(
        waveform.numpy().tobytes(),
        frame_rate=sample_rate,
        sample_width=waveform.numpy().dtype.itemsize,
        channels=len(waveform)
    )

    cropped_audio = audio_segment[:duration_ms]
    cropped_audio.export(output_file, format="mp3")

# Call the function to crop and save the audio file
input_audio_file = "file.mp3"  # Replace with the path to your audio file
output_audio_file = "file.mp3"
crop_and_save_audio(input_audio_file, output_audio_file, duration=5000)  # Crop to 5 seconds (5000 milliseconds)



In [3]:


# Load the trained ML classifier
model_path = 'model_gsearch_svm.pkl'
model = joblib.load(model_path)

# Load the label encoder
label_encoder_path = 'label_encoder_gsearch_svm.pkl'
label_encoder = joblib.load(label_encoder_path)

# Paths to VGGish model checkpoint and PCA parameters
checkpoint_path = 'vggish_model.ckpt'
pca_params_path = 'vggish_pca_params.npz'

run_name = 'demo_run'


def predict_language(audio_file):
    # Initialize the FeatureExtractor with the necessary arguments
    feature_extractor = FeatureExtractor(checkpoint_path, pca_params_path, run_name)
    feature_extractor.load_model()

    # Extract embeddings from the audio file using the FeatureExtractor
    embeddings = feature_extractor.extract_features(audio_file)
    if embeddings is not None:
        print("Shape of embeddings:", embeddings.shape)

        # Shape of the target embedding
        target_shape = (10, 640)
        embedding_shape = embeddings.shape
        # Repeat the sample embedding to match the target shape
        repeated_embeddings = np.tile(embeddings, (target_shape[0] // embedding_shape[0], target_shape[1] // embedding_shape[1]))

        print("Repeated Embeddings:")
        print(repeated_embeddings)
        print("Shape of repeated embeddings:", repeated_embeddings.shape)

        # Predict the language label
        predictions = model.predict(repeated_embeddings)

        language_labels = label_encoder.inverse_transform(predictions)
        #print(label_encoder.classes_)
        return language_labels
    else:
        print("Error: Embeddings could not be extracted from the audio file.")
        return None
language_label = predict_language("file.mp3")
language_label= language_label.tolist()
print("Predicted Language Labels: ", language_label)
print("Predicted Label after Majority Voting: ",max(set(language_label), key = language_label.count))


Instructions for updating:
non-resource variables are not supported in the long term


Error extracting features from file.mp3: Error opening 'file.mp3': System error.
Error: Embeddings could not be extracted from the audio file.


AttributeError: ignored